In [1]:
from langchain_community.document_loaders.pdf import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [2]:
caminhos = [
    "files/apostila.pdf",
    "files/LLM.pdf"
    ]

paginas = []

for caminho in caminhos:
    loader = PyPDFLoader(caminho)
    paginas.extend(loader.load())

recur_split = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    separators=["\n\n", "\n", ".", " ", ""]
)

documents = recur_split.split_documents(paginas)

Ignoring wrong pointing object 16 0 (offset 0)
Ignoring wrong pointing object 18 0 (offset 0)
Ignoring wrong pointing object 20 0 (offset 0)
Ignoring wrong pointing object 22 0 (offset 0)
Ignoring wrong pointing object 42 0 (offset 0)
Ignoring wrong pointing object 50 0 (offset 0)
Ignoring wrong pointing object 52 0 (offset 0)
Ignoring wrong pointing object 54 0 (offset 0)
Ignoring wrong pointing object 56 0 (offset 0)
Ignoring wrong pointing object 58 0 (offset 0)
Ignoring wrong pointing object 70 0 (offset 0)
Ignoring wrong pointing object 72 0 (offset 0)
Ignoring wrong pointing object 89 0 (offset 0)
Ignoring wrong pointing object 91 0 (offset 0)
Ignoring wrong pointing object 103 0 (offset 0)
Ignoring wrong pointing object 108 0 (offset 0)
Ignoring wrong pointing object 149 0 (offset 0)
Ignoring wrong pointing object 155 0 (offset 0)
Ignoring wrong pointing object 158 0 (offset 0)
Ignoring wrong pointing object 160 0 (offset 0)
Ignoring wrong pointing object 163 0 (offset 0)
Ignori

In [3]:
for i, doc in enumerate(documents):
    doc.metadata['source'] = doc.metadata['source'].replace('files/', '')
    doc.metadata['doc_id'] = i

In [5]:
documents[2].metadata

{'source': 'apostila.pdf', 'page': 1, 'doc_id': 2}

In [6]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores.chroma import Chroma

embeddings_model = OpenAIEmbeddings()

In [7]:
diretorio = 'files/chroma_retrival_bd'

vectordb = Chroma.from_documents(
    documents=documents,
    embedding=embeddings_model,
    persist_directory=diretorio
)

### Semantic Search

In [8]:
pergunta = "O que é LLM?"
docs = vectordb.similarity_search(pergunta, k=3)
for doc in docs:
    print(doc.page_content)
    print(f"========{doc.metadata}\n")

. Se você quiser se aprofundar um pouco mais e expandir seus conhecimentos e compreensão dos fundamentos dos LLMs, recomendamos conferir nosso curso sobre LLMs. Você aprenderá como desenvolver aplicativos prontos para produção com LLMs e se aprofundará na teoria por trás dos modelos de fundação
========{'doc_id': 141, 'page': 7, 'source': 'LLM.pdf'}

PARTE  1 Introdução   Definição de LLM (tradução livre: grandes modelos de linguagem) LLMs são sistemas de IA desenvolvidos para processar e analisar enormes quantidades de dados de linguagem natural e, em seguida, usar essas informações para gerar respostas às solicitações dos usuários
========{'doc_id': 101, 'page': 1, 'source': 'LLM.pdf'}

. Conclusão e diretrizes gerais Em última análise, cada organização terá desafios únicos a superar, e não existe uma abordagem única para os LLMs. À medida que o mundo se torna mais orientado a dados, tudo, incluindo os LLMs, dependerá de uma base sólida de dados. Os LLMs são ferramentas incríveis, ma

### Max Margina Relevance

In [9]:
pergunta = "O que é LLM?"
docs = vectordb.max_marginal_relevance_search(pergunta, k=3, fetch_k=10)
for doc in docs:
    print(doc.page_content)
    print(f"========{doc.metadata}\n")

. Se você quiser se aprofundar um pouco mais e expandir seus conhecimentos e compreensão dos fundamentos dos LLMs, recomendamos conferir nosso curso sobre LLMs. Você aprenderá como desenvolver aplicativos prontos para produção com LLMs e se aprofundará na teoria por trás dos modelos de fundação
========{'doc_id': 141, 'page': 7, 'source': 'LLM.pdf'}

E-BOOK Um guia compacto sobre Large Language Models (LLM)
========{'doc_id': 99, 'page': 0, 'source': 'LLM.pdf'}

.   ANÁLISE DE SENTIMENTO  Frequentemente, uma tarefa difícil de quantificar, os LLMs podem ajudar a analisar emoções e opiniões a partir de um texto. Isso pode ajudar as organizações a coletarem os dados e o feedback necessários para melhorar a satisfação dos clientes
========{'doc_id': 120, 'page': 4, 'source': 'LLM.pdf'}



### FIltragem

In [10]:
pergunta = "O que a apostila de LLM fala sobre a OpenAI e o ChatGPT?"

docs = vectordb.similarity_search(
    pergunta,
    k=3,
    filter={"source": "LLM.pdf"}
)
for doc in docs:
    print(doc.page_content)
    print(f"========{doc.metadata}\n")

.   Serviços proprietários Como o primeiro serviço amplamente disponível alimentado por LLM, o ChatGPT da OpenAI foi o catalisador explosivo que trouxe os LLMs para o mainstream. O ChatGPT fornece uma interface de usuário (ou API) em que os usuários podem enviar prompts para muitos modelos (GPT-3.5, GPT-4 e outros) e geralmente obter uma resposta rápida
========{'doc_id': 126, 'page': 5, 'source': 'LLM.pdf'}

.  2020 A OpenAI lançou o GPT-3, que se tornou o maior modelo com 175 bilhões de parâmetros e estabeleceu um novo referencial de desempenho para tarefas relacionadas à linguagem.  2022 O ChatGPT foi lançado, transformando o GPT-3 e modelos semelhantes em um serviço amplamente acessível aos usuários por meio de uma interface web, o que iniciou um aumento significativo na conscientização pública sobre LLMs e IA generativa
========{'doc_id': 108, 'page': 2, 'source': 'LLM.pdf'}

. MAIOR ACESSIBILIDADE  O lançamento do ChatGPT abriu as portas para qualquer pessoa com acesso à internet

In [11]:
pergunta = 'O que a apostila de LLM fala sobre a OpenAI e o ChatGPT?'

docs = vectordb.similarity_search(
    pergunta, 
    k=3,
    filter={'$and':
            [{'source': {'$in': ['LLM.pdf']}},
            {'page': {'$in': [3, 4, 5, 6]}}],
            })
for doc in docs:
    print(doc.page_content)
    print(f'==========={doc.metadata}\n\n')

.   Serviços proprietários Como o primeiro serviço amplamente disponível alimentado por LLM, o ChatGPT da OpenAI foi o catalisador explosivo que trouxe os LLMs para o mainstream. O ChatGPT fornece uma interface de usuário (ou API) em que os usuários podem enviar prompts para muitos modelos (GPT-3.5, GPT-4 e outros) e geralmente obter uma resposta rápida
==========={'doc_id': 126, 'page': 5, 'source': 'LLM.pdf'}


. MAIOR ACESSIBILIDADE  O lançamento do ChatGPT abriu as portas para qualquer pessoa com acesso à internet interagir com um dos LLMs mais avançados por meio de uma interface web simples. Isso trouxe os impressionantes avanços dos LLMs para o centro das atenções, uma vez que anteriormente esses modelos mais poderosos estavam disponíveis apenas para pesquisadores com recursos significativos e conhecimento técnico profundo
==========={'doc_id': 114, 'page': 3, 'source': 'LLM.pdf'}


. Comunidades como a Hugging Face reúnem centenas de milhares de modelos de contribuidores que pod

### LLM Aided Retrieval

In [12]:
from langchain_openai.llms import OpenAI
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.chains.query_constructor.schema import AttributeInfo

In [13]:
metadata_info = [
    AttributeInfo(
        name='source',
        description='Nome da apostila de onde o texto original foi retirado. Pode ser "apostila.pdf" ou "LLM.pdf".',
        type='string'
    ),
    AttributeInfo(
        name='page',
        description='A página da apostila de onde o texto foi extraído. Número da página.',
        type='integer'
    ),
]


In [14]:
document_description = 'Apostilas de informações'
llm = OpenAI()
retriever = SelfQueryRetriever.from_llm(
    llm,
    vectordb,
    document_description,
    metadata_info,
    verbose=True
)

In [15]:
pergunta = 'O que a apostila de LLM fala sobre a OpenAI, ChatGPT e Hugging Face?'

docs = retriever.get_relevant_documents(pergunta)
for doc in docs:
    print(doc.page_content)
    print(f'==========={doc.metadata}\n\n')

d:\Udemy\2025\langchain\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


. Comunidades como a Hugging Face reúnem centenas de milhares de modelos de contribuidores que podem ajudar a resolver muitos casos de uso específicos, como geração de texto, resumo e classificação. A comunidade de código aberto está rapidamente alcançando o desempenho dos modelos proprietários, mas ainda não conseguiu igualar o desempenho de algo como o GPT-4.
==========={'doc_id': 131, 'page': 5, 'source': 'LLM.pdf'}


.  2020 A OpenAI lançou o GPT-3, que se tornou o maior modelo com 175 bilhões de parâmetros e estabeleceu um novo referencial de desempenho para tarefas relacionadas à linguagem.  2022 O ChatGPT foi lançado, transformando o GPT-3 e modelos semelhantes em um serviço amplamente acessível aos usuários por meio de uma interface web, o que iniciou um aumento significativo na conscientização pública sobre LLMs e IA generativa
==========={'doc_id': 108, 'page': 2, 'source': 'LLM.pdf'}


.   Serviços proprietários Como o primeiro serviço amplamente disponível alimentado por LL